In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('PQ-300.xlsx', usecols = 'A:F', header = 0, nrows = 12)

df = (
    df
    .assign(Column1 = lambda d: d['Column1'].ffill())
    .assign(FactoryCode = lambda d: d['Column1'].str.extract(r'Factory\s*-\s*(\w+)').ffill())
)
df

,Column1,Column2,Column3,Column4,Column5,Column6,FactoryCode
0,Factory - A,NaN,Q1,Q2,Q3,Q4,A
1,Project 1,Budget,358,697,954,165,A
2,Project 1,Actual,989,126,877,686,A
3,Project 2,Budget,906,572,328,101,A
4,Project 2,Actual,976,284,248,183,A
5,Factory - B,NaN,NaN,NaN,NaN,NaN,B
6,Project 1,Budget,473,513,385,418,B
7,Project 1,Actual,450,357,952,249,B
8,Project 2,Budget,790,689,135,840,B
9,Project 2,Actual,102,304,747,890,B


In [3]:
temp = []
for n, group in df.groupby('FactoryCode'):
    
    temp_df = group[1:]
    temp_df.columns = ['Project', 'Type', 'Q1', 'Q2', 'Q3', 'Q4', 'Factory']
    
    unpivot = temp_df.melt(
        id_vars = ['Factory', 'Project', 'Type'], 
        value_vars = ['Q1', 'Q2', 'Q3', 'Q4'], 
        var_name = 'Quarter', 
        value_name = 'A'
    )
    
    pivot = unpivot.pivot_table(
        index = ['Factory', 'Project', 'Quarter'], 
        values = 'A', columns = 'Type', 
        aggfunc = 'sum'
    )

    pivot = pivot.reset_index()
    temp.append(pivot)

temp

[Type Factory    Project Quarter Actual Budget
 0          A  Project 1      Q1    989    358
 1          A  Project 1      Q2    126    697
 2          A  Project 1      Q3    877    954
 3          A  Project 1      Q4    686    165
 4          A  Project 2      Q1    976    906
 5          A  Project 2      Q2    284    572
 6          A  Project 2      Q3    248    328
 7          A  Project 2      Q4    183    101,
 Type Factory    Project Quarter Actual Budget
 0          B  Project 1      Q1    450    473
 1          B  Project 1      Q2    357    513
 2          B  Project 1      Q3    952    385
 3          B  Project 1      Q4    249    418
 4          B  Project 2      Q1    102    790
 5          B  Project 2      Q2    304    689
 6          B  Project 2      Q3    747    135
 7          B  Project 2      Q4    890    840
 8          B  Project 3      Q1    597    304
 9          B  Project 3      Q2    883    962
 10         B  Project 3      Q3    634    497
 11         

In [4]:
df = (
    pd.concat(temp, ignore_index = True)
    .assign(
        ProjectSort = lambda d: d['Project'].str.extract(r'(\d+)'),
        QuarterSort = lambda d: d['Quarter'].str.extract(r'(\d+)')
    )
    .sort_values(['Factory', 'ProjectSort', 'QuarterSort'])
    .drop(columns = ['ProjectSort', 'QuarterSort'])
    .get(['Factory', 'Quarter', 'Project', 'Budget', 'Actual'])
)

df.columns.name = None
df

,Factory,Quarter,Project,Budget,Actual
0,A,Q1,Project 1,358,989
1,A,Q2,Project 1,697,126
2,A,Q3,Project 1,954,877
3,A,Q4,Project 1,165,686
4,A,Q1,Project 2,906,976
5,A,Q2,Project 2,572,284
6,A,Q3,Project 2,328,248
7,A,Q4,Project 2,101,183
8,B,Q1,Project 1,473,450
9,B,Q2,Project 1,513,357
